In [1]:
import pyradiosky
import numpy as np
import astropy.units as units
from astropy.units import Quantity
from astropy.coordinates import Latitude, Longitude, SkyCoord
import pyuvdata

In [2]:
catalog_savepath = "/Users/ruby/Astro/polarized_source_sims_Feb2022/polarized_source.skyh5"

zen_ra = 0.
zen_dec = -26.7

source_dist_from_zenith = 10.
source_angle = 135.

In [3]:
zen_theta = 90.-zen_dec

In [4]:
source_rotated_frame_coords = np.array([
    np.sin(np.radians(source_dist_from_zenith))*np.cos(np.radians(source_angle)),
    np.sin(np.radians(source_dist_from_zenith))*np.sin(np.radians(source_angle)),
    np.cos(np.radians(source_dist_from_zenith))
])
rot_mat = np.array([
    [np.cos(np.radians(zen_theta)), 0, -np.sin(np.radians(zen_theta))],
    [0., 1., 0.],
    [np.sin(np.radians(zen_theta)), 0, np.cos(np.radians(zen_theta))],
], dtype=float)
rot_mat_inv = np.linalg.inv(rot_mat)

In [5]:
source_coords = np.matmul(rot_mat_inv, source_rotated_frame_coords)

In [6]:
source_theta = np.degrees(np.arccos(source_coords[2]))
source_phi = np.degrees(np.arctan(source_coords[1]/source_coords[0]))

In [7]:
source_dec = 90.-source_theta
source_ra = 360.+zen_ra-source_phi

In [8]:
print(source_ra)
print(source_dec)

352.51827221039343
-19.438674107505264


In [9]:
nsources = 1
nfreqs = 1
flux = 10.
stokes_q_pol_fraction = 0.5
freq_mhz = 182.
spec_index = 0.
cat_stokes = Quantity(np.zeros((4, nfreqs, nsources), dtype=float), "Jy")
cat_stokes[0, 0, 0] = flux * units.Jy
cat_stokes[1, 0, 0] = stokes_q_pol_fraction * flux * units.Jy
cat_spectral_index = np.full(nsources, spec_index)
cat_name = np.array(["pol_source"])
cat_RA = np.array([source_ra])
cat_Dec = np.array([source_dec])

catalog = pyradiosky.SkyModel(
    name=cat_name,
    ra=Longitude(cat_RA, units.deg),
    dec=Latitude(cat_Dec, units.deg),
    stokes=cat_stokes,
    spectral_type="spectral_index",
    reference_frequency=Quantity(np.full(nsources, freq_mhz * 1e6), "hertz"),
    spectral_index=cat_spectral_index,
)
if not catalog.check():
    print("ERROR: Catalog check failed.")

catalog.write_skyh5(catalog_savepath)

In [10]:
# Write beam model as beamfits file
# Don't actually do this because there isn't enough space on my laptop
#mwa_beam = pyuvdata.UVBeam()
#mwa_beam.read_mwa_beam("/Users/ruby/Astro/polarized_source_sims_Feb2022/mwa_full_embedded_element_pattern.h5")
#mwa_beam.write_beamfits("/Users/ruby/Astro/polarized_source_sims_Feb2022/mwa_full_embedded_element_pattern.fits")

In [11]:
# Also tough for my laptop, run on Wario instead
#uv = pyuvdata.UVData()
#uv.read_uvfits("/users/ruby/Astro/1061316296.uvfits")
#pyuvsim.uvdata_to_config_file(uv, path_out="/Users/ruby/Astro/polarized_source_sims_Feb2022/")
#pyuvsim.uvdata_to_telescope_config(uv, path_out="/Users/ruby/Astro/polarized_source_sims_Feb2022/")

In [12]:
(2456528.2545138896-2456528.2532407343)*(24*60*60)/56.

1.964296613420759